In [1]:
from typing import Any

import numpy as np
import numpy.typing as npt

In [2]:
data_113 = np.loadtxt('dataset/20151026_113_labeled.txt')
data_114 = np.loadtxt('dataset/20151026_114_labeled.txt')
data = np.concatenate((data_113, data_114), axis=0)

features = data[:, :6]
labels = data[:, 6].astype(np.int8)

print(features.shape)
print(labels.shape)

(600, 6)
(600,)


In [3]:
def calc_entropy(labels: npt.NDArray[np.int8]) -> float:
    '''
    Calculate the entropy of the labels.
    '''
    entropy = 0

    for count in np.unique(labels, return_counts=True)[1]:
        # H(D) = -Σ(|Ck| / |D| * log2(|Ck| / |D|))
        prob = count / len(labels)
        entropy -= prob * np.log2(prob)

    return entropy


def calc_info_gain(features: npt.NDArray[np.float64], labels: npt.NDArray[np.int8], split_value: float) -> float:
    '''
    Calculate the information gain of the features and labels.
    '''
    # Split the labels by the split value.
    left_labels = labels[features < split_value]
    right_labels = labels[features >= split_value]

    entropy = calc_entropy(labels)
    left_entropy = calc_entropy(left_labels)
    right_entropy = calc_entropy(right_labels)

    left_prob = len(left_labels) / len(labels)
    right_prob = len(right_labels) / len(labels)

    # H(D|A) = Σ(|Di| / |D| * H(Di))
    conditional_entropy = left_prob * left_entropy + right_prob * right_entropy

    # G(D|A) = H(D) - H(D|A)
    info_gain = entropy - conditional_entropy

    return info_gain


def choose_best_feature(features: npt.NDArray[np.float64], labels: npt.NDArray[np.int8]) -> tuple[int, float]:
    '''
    Choose the best feature and split value to split the dataset.
    '''
    best_feature_index = -1
    best_split_value = -1
    best_info_gain = 0

    # Iterate over all features
    for feature_index in range(features.shape[1]):
        feature = features[:, feature_index]

        # Iterate over all values of the feature
        for value in feature:
            info_gain = calc_info_gain(feature, labels, value)

            # Update the best feature and split value
            if info_gain > best_info_gain:
                best_feature_index = feature_index
                best_split_value = value
                best_info_gain = info_gain

    return best_feature_index, best_split_value

In [4]:
def build_tree(features: npt.NDArray[np.float64], labels: npt.NDArray[np.int8]) -> dict[str, Any]:
    '''
    Build the decision tree recursively.
    '''
    tree: dict[str, Any] = {}

    # If all the labels are the same, end the recursion.
    if np.all(labels == labels[0]):
        tree['label'] = labels[0]
        return tree

    feature_index, split_value = choose_best_feature(features, labels)

    tree['feature_index'] = feature_index
    tree['split_value'] = split_value

    feature = features[:, feature_index]
    tree['left'] = build_tree(features[feature < split_value],
                              labels[feature < split_value])
    tree['right'] = build_tree(features[feature >= split_value],
                               labels[feature >= split_value])
    return tree

In [5]:
tree = build_tree(features, labels)

display(tree)

{'feature_index': 0,
 'split_value': 270.79117585820137,
 'left': {'feature_index': 4,
  'split_value': 0.2265714231505857,
  'left': {'feature_index': 4,
   'split_value': -0.3938333982986771,
   'left': {'feature_index': 5,
    'split_value': -0.7126152948127243,
    'left': {'feature_index': 0,
     'split_value': 112.38079799254311,
     'left': {'feature_index': 0,
      'split_value': 99.24425716472952,
      'left': {'label': 1},
      'right': {'label': 2}},
     'right': {'feature_index': 5,
      'split_value': -1.1411183120960675,
      'left': {'feature_index': 5,
       'split_value': -24.083641275755816,
       'left': {'label': 1},
       'right': {'label': 0}},
      'right': {'feature_index': 0,
       'split_value': 232.16245743708612,
       'left': {'label': 0},
       'right': {'label': 1}}}},
    'right': {'feature_index': 0,
     'split_value': 257.30675332208466,
     'left': {'label': 0},
     'right': {'feature_index': 0,
      'split_value': 261.6240555249553